In [1]:
import torch
import torchvision

print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("CUDA available?:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

torch: 2.4.1+cu124
torchvision: 0.19.1+cu124
CUDA available?: True
GPU: NVIDIA A40


In [2]:
# 0. (필요 시) 패키지 설치
!pip install transformers>=4.40.0 pillow


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
# 1. 라이브러리 임포트
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

In [4]:
# 2. 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [5]:
# 3. 모델과 프로세서 로드 (VQA 전용)
from transformers import BlipProcessor, BlipForQuestionAnswering

model_name_vqa = "Salesforce/blip-vqa-base"

processor_vqa = BlipProcessor.from_pretrained(model_name_vqa, use_fast=True)
model_vqa = BlipForQuestionAnswering.from_pretrained(
    model_name_vqa,
    use_safetensors=True
).to(device)


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [6]:
# 4. 이미지 및 질문 준비
image_path = "Golden-Retriever.jpg"  # 본인 환경에 맞게 경로 수정
question = "What is the dog doing?"     # 예시 질문
image = Image.open(image_path).convert("RGB")

In [7]:
# 5. 전처리: 이미지와 질문을 함께 처리
inputs = processor_vqa(
    images=image,
    text=question,
    return_tensors="pt"
).to(device)

In [8]:
# 6. 답변 생성
with torch.no_grad():
    outputs = model_vqa.generate(
        **inputs,
        max_length=16,      # 답변 최대 토큰 수
        num_beams=4,
        early_stopping=True
    )

In [9]:
# 7. 결과 디코딩 및 출력
answer = processor_vqa.decode(outputs[0], skip_special_tokens=True)
print(f"🔹 Question: {question}")
print(f"🔹 Answer:   {answer}")

🔹 Question: What is the dog doing?
🔹 Answer:   sitting on log
